IMDB Data Import - need to import using SQL 

In [2]:
#import all programs needed
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
#establish connection to imdb database

conn = sqlite3.connect('Phase 1 Project Data/im.db')
cursor_obj = conn.cursor()

In [5]:
#create movie_basics dataframe and examine the data 
movie_basics_df = pd.read_sql('''
    SELECT *
    FROM movie_basics
    ''', conn)

movie_basics_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146144 entries, 0 to 146143
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   movie_id         146144 non-null  object 
 1   primary_title    146144 non-null  object 
 2   original_title   146123 non-null  object 
 3   start_year       146144 non-null  int64  
 4   runtime_minutes  114405 non-null  float64
 5   genres           140736 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 6.7+ MB


Notes on movie_basics: Missing data in the original_title column (<1%), genres (4%) and runtime_minutes (21%). Nulls in title and genre are better to drop, while runtime should be inputed based on mean or median. 

Data types make sense 

Genres is a string containing multiple genres. Needs to be split into multiple columns. Possibly use a primary/secondary/tertiary genre structure based on the order it is listed in the string 

In [13]:
#create movie_ratings dataframe and examine the data
movie_ratings_df = pd.read_sql('''
    SELECT *
    FROM movie_ratings
    ''', conn)

movie_ratings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73856 entries, 0 to 73855
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   movie_id       73856 non-null  object 
 1   averagerating  73856 non-null  float64
 2   numvotes       73856 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 1.7+ MB


Notes on movie_rating dataframe: only two columns. Wide range in the number of votes, possibly need to create a minimum vote count to be included in the ratings analysis (mean - 3*std).

Possible outliers included. The minimum average rating is more than 3 standard deviations away from the mean. The std of the numvotes column is an order of magnitude greater than the mean.

Smaller than the movie_basics dataframe so it would make sense to merge left onto the basics df 

In [15]:
#create director dataframe and examine data
director_df = pd.read_sql('''
    SELECT *
    FROM directors
    ''', conn)

director_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 291174 entries, 0 to 291173
Data columns (total 2 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   movie_id   291174 non-null  object
 1   person_id  291174 non-null  object
dtypes: object(2)
memory usage: 4.4+ MB


Only contains the person_id for the director, if I want names then I will have to merge with the persons dataframe. Otherwise looks good with no null values 

In [16]:
#create persons dataframe and examine data
persons_df = pd.read_sql('''
    SELECT *
    FROM persons
    ''', conn)

persons_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 606648 entries, 0 to 606647
Data columns (total 5 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   person_id           606648 non-null  object 
 1   primary_name        606648 non-null  object 
 2   birth_year          82736 non-null   float64
 3   death_year          6783 non-null    float64
 4   primary_profession  555308 non-null  object 
dtypes: float64(2), object(3)
memory usage: 23.1+ MB


Need to concat with directors df in order to create a full list of movie ids with director names 

In [34]:
#merge director and persons dataframes to create a unified df of director credits with names 

director_persons_df = pd.merge(director_df, persons_df, how = 'left', on = 'person_id')
director_persons_df.info()

#remove duplicates before merging with movie_basics_ratings

director_persons_df.drop_duplicates(subset = 'movie_id', inplace = True)
director_persons_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 291174 entries, 0 to 291173
Data columns (total 6 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   movie_id            291174 non-null  object 
 1   person_id           291174 non-null  object 
 2   primary_name        291171 non-null  object 
 3   birth_year          68608 non-null   float64
 4   death_year          1738 non-null    float64
 5   primary_profession  290187 non-null  object 
dtypes: float64(2), object(4)
memory usage: 15.6+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 140417 entries, 0 to 291173
Data columns (total 6 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   movie_id            140417 non-null  object 
 1   person_id           140417 non-null  object 
 2   primary_name        140416 non-null  object 
 3   birth_year          30609 non-null   float64
 4   death_year          85

In [31]:
#merge movie ratings and movie basics dfs together 

movie_basics_ratings_df = pd.merge(movie_basics_df, movie_ratings_df, how ='left', on = 'movie_id')
movie_basics_ratings_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 146144 entries, 0 to 146143
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   movie_id         146144 non-null  object 
 1   primary_title    146144 non-null  object 
 2   original_title   146123 non-null  object 
 3   start_year       146144 non-null  int64  
 4   runtime_minutes  114405 non-null  float64
 5   genres           140736 non-null  object 
 6   averagerating    73856 non-null   float64
 7   numvotes         73856 non-null   float64
dtypes: float64(3), int64(1), object(4)
memory usage: 10.0+ MB


In [47]:
#create final IMDB database by merging movie_basics_ratings_df left onto director_persons_df 

imdb_df = pd.merge(director_persons_df, movie_basics_ratings_df, how = 'left', on = 'movie_id')
imdb_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 140417 entries, 0 to 140416
Data columns (total 13 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   movie_id            140417 non-null  object 
 1   person_id           140417 non-null  object 
 2   primary_name        140416 non-null  object 
 3   birth_year          30609 non-null   float64
 4   death_year          856 non-null     float64
 5   primary_profession  139887 non-null  object 
 6   primary_title       140417 non-null  object 
 7   original_title      140415 non-null  object 
 8   start_year          140417 non-null  int64  
 9   runtime_minutes     112020 non-null  float64
 10  genres              137239 non-null  object 
 11  averagerating       73104 non-null   float64
 12  numvotes            73104 non-null   float64
dtypes: float64(5), int64(1), object(7)
memory usage: 15.0+ MB


In [59]:
#clean up the IMDB dataframe
#rename primary_name to director_name 
imdb_df.rename(columns = {'primary_name':'director_name'}, inplace = True)

#check where primary title != original title 

imdb_df.loc[(imdb_df['primary_title'] == imdb_df['original_title'])]


,movie_id,person_id,director_name,birth_year,death_year,primary_profession,primary_title,original_title,start_year,runtime_minutes,genres,averagerating,numvotes
0,tt0285252,nm0899854,Tony Vitale,1964.0,NaN,"producer,director,writer",Life's a Beach,Life's a Beach,2012,100.0,Comedy,3.9,219.0
1,tt0462036,nm1940585,Bill Haley,NaN,NaN,"director,writer,producer",Steve Phoenix: The Untold Story,Steve Phoenix: The Untold Story,2012,110.0,Drama,5.5,18.0
2,tt0835418,nm0151540,Jay Chandrasekhar,1968.0,NaN,"director,actor,writer",The Babymakers,The Babymakers,2012,95.0,Comedy,5.0,8147.0
3,tt0878654,nm0089502,Albert Pyun,1954.0,NaN,"director,writer,producer",Bulletface,Bulletface,2010,82.0,Thriller,5.8,875.0
4,tt0879859,nm2416460,Eric Manchester,NaN,NaN,"director,writer",Torn,Torn,2010,NaN,Thriller,6.8,21.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
140412,tt8999892,nm10122247,C. Damon Adcock,NaN,NaN,None,Dumpster Fire: A Time Of Current Times,Dumpster Fire: A Time Of Current Times,2019,108.0,Comedy,NaN,NaN
140413,tt8999974,nm10122357,Daysi Burbano,NaN,NaN,"director,writer,cinematographer",Madre Luna,Madre Luna,2018,80.0,Documentary,NaN,NaN
140414,tt9001390,nm6711477,Bernard Lessa,NaN,NaN,"director,writer,cinematographer",The woman and the river,The woman and the river,2017,88.0,"Drama,Mystery",NaN,NaN
140415,tt9001494,nm10123242,Tate Nova,NaN,NaN,"director,producer",Stone Mountain Georgia 08.28.18,Stone Mountain Georgia 08.28.18,2018,5.0,"Documentary,History,News",NaN,NaN
